In [11]:
# @title Setup
# --- Setup (run this cell) ---
!pip -q install freesasa biopython py3Dmol ipywidgets

# Colab widget support (safe to run elsewhere too)
try:
    from google.colab import output
    output.enable_custom_widget_manager()
except Exception:
    pass

In [12]:
# @title Imports
import io
import os
import re
import math
import textwrap
import tempfile
import traceback
from dataclasses import dataclass
from typing import Dict, List, Tuple, Optional, Any
from google.colab import files

import numpy as np
import pandas as pd

import freesasa
from Bio.PDB import PDBParser, PDBIO, Select
from Bio.PDB.Structure import Structure
from Bio.PDB.Model import Model
from Bio.PDB.Chain import Chain
from Bio.PDB.Residue import Residue
from Bio.SeqUtils.ProtParam import ProteinAnalysis

import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output

import py3Dmol

In [13]:
# @title Helper Functions

STANDARD_AA3 = {
    "ALA","ARG","ASN","ASP","CYS","GLN","GLU","GLY","HIS","ILE",
    "LEU","LYS","MET","PHE","PRO","SER","THR","TRP","TYR","VAL"
}

AA3_TO_AA1 = {
    "ALA":"A","ARG":"R","ASN":"N","ASP":"D","CYS":"C","GLN":"Q","GLU":"E","GLY":"G","HIS":"H","ILE":"I",
    "LEU":"L","LYS":"K","MET":"M","PHE":"F","PRO":"P","SER":"S","THR":"T","TRP":"W","TYR":"Y","VAL":"V"
}
_CANONICAL_AA1 = set("ACDEFGHIKLMNPQRSTVWY")

def _safe_float(x, default=0.0):
    try:
        if x is None:
            return default
        return float(x)
    except Exception:
        return default

def _select_best_altloc(atom):
    """For disordered atoms, select the altloc with highest occupancy (fallback: first)."""
    if not atom.is_disordered():
        return atom
    best_id = None
    best_occ = -1.0
    for alt_id, child in atom.child_dict.items():
        occ = _safe_float(child.get_occupancy(), default=0.0)
        if occ > best_occ:
            best_occ = occ
            best_id = alt_id
    if best_id is None:
        best_id = list(atom.child_dict.keys())[0]
    atom.disordered_select(best_id)
    return atom

def _normalize_residue(res: Residue):
    """Normalize residue names (e.g. MSE->MET) and altloc selection."""
    if res.get_resname().strip() == "MSE":
        res.resname = "MET"
        if "SE" in res:
            res["SE"].set_name("SD")
            atom = res["SE"]
            del res.child_dict["SE"]
            res.add(atom)
    for atom in list(res.get_atoms()):
        if atom.is_disordered():
            _select_best_altloc(atom)

def _first_protein_chain(structure: Structure) -> Optional[str]:
    for model in structure:
        for chain in model:
            if any((res.get_resname().strip() in STANDARD_AA3) for res in chain.get_residues()):
                return chain.id
    return None

class _ChainSelect(Select):
    def __init__(self, chain_id: str):
        self.chain_id = chain_id

    def accept_chain(self, chain: Chain):
        return 1 if chain.id == self.chain_id else 0

    def accept_residue(self, residue: Residue):
        return 1 if residue.get_resname().strip() in STANDARD_AA3 else 0

def parse_pdb_to_structure(pdb_path: str) -> Structure:
    parser = PDBParser(QUIET=True)
    structure_id = os.path.basename(pdb_path)
    return parser.get_structure(structure_id, pdb_path)

def write_clean_chain_pdb(structure: Structure, chain_id: str, out_path: str) -> None:
    for model in structure:
        for chain in model:
            for res in chain.get_residues():
                _normalize_residue(res)
    io_ = PDBIO()
    io_.set_structure(structure)
    io_.save(out_path, select=_ChainSelect(chain_id))

@dataclass
class MetricsResult:
    filename: str
    chain_id: str
    total_sasa: float
    hydrophobic_sasa: float
    hydrophobic_fraction: float
    predicted_pI: float

def compute_sasa_with_freesasa(clean_pdb_path: str) -> Tuple[float, pd.DataFrame]:
    """Returns total SASA and per-residue SASA table for the PDB (Å²)."""
    fs_struct = freesasa.Structure(clean_pdb_path)
    result = freesasa.calc(fs_struct)
    total = float(result.totalArea())

    res_areas = result.residueAreas()
    rows = []
    for chain_id, chain_dict in res_areas.items():
        for res_key, area_obj in chain_dict.items():
            area_total = getattr(area_obj, "total", None)
            if area_total is None:
                area_total = area_obj.get("total", np.nan) if isinstance(area_obj, dict) else np.nan

            resnum = None
            icode = ""
            resname = ""
            if isinstance(res_key, tuple):
                if len(res_key) >= 1: resnum = res_key[0]
                if len(res_key) >= 2: icode = res_key[1] if res_key[1] is not None else ""
                if len(res_key) >= 3: resname = res_key[2] if res_key[2] is not None else ""
            else:
                resnum = res_key
                resname = getattr(area_obj, "residueName", "") if hasattr(area_obj, "residueName") else ""

            rows.append({
                "chain": str(chain_id),
                "resnum": resnum,
                "icode": str(icode).strip(),
                "resname3": str(resname).strip(),
                "sasa": float(area_total) if area_total is not None else np.nan,
            })

    df = pd.DataFrame(rows)
    return total, df

def _augment_per_residue_df(clean_pdb_path: str, per_res_df: pd.DataFrame, chain_id: str, hydrophobic_aa1: set) -> pd.DataFrame:
    """Ensure residue names/AA1 are present; add is_hydrophobic; sort."""
    structure = parse_pdb_to_structure(clean_pdb_path)

    resname_map = {}
    for model in structure:
        if chain_id not in model:
            continue
        for res in model[chain_id]:
            if res.get_resname().strip() in STANDARD_AA3:
                rid = res.get_id()
                resnum = int(rid[1])
                icode = rid[2].strip() if isinstance(rid[2], str) else ""
                resname_map[(resnum, icode)] = res.get_resname().strip()

    df = per_res_df.copy()
    df = df[df["chain"] == chain_id].copy()
    df["icode"] = df["icode"].fillna("").astype(str).str.strip()
    df["resnum_num"] = pd.to_numeric(df["resnum"], errors="coerce")

    # Fill missing resname3 if FreeSASA did not provide it
    def _fill_resname(row):
        if pd.notna(row.get("resname3")) and str(row.get("resname3")).strip():
            return str(row.get("resname3")).strip()
        if pd.isna(row["resnum_num"]):
            return ""
        return resname_map.get((int(row["resnum_num"]), row["icode"]), "")

    df["resname3"] = df.apply(_fill_resname, axis=1)
    df["aa1"] = df["resname3"].map(AA3_TO_AA1).fillna("?")
    df["is_hydrophobic"] = df["aa1"].isin(hydrophobic_aa1)

    df = df.sort_values(["resnum_num", "icode"], kind="mergesort").drop(columns=["resnum_num"]).reset_index(drop=True)
    return df

def _predict_pi_from_df(per_res_df: pd.DataFrame) -> float:
    if per_res_df is None or len(per_res_df) == 0:
        return float("nan")
    seq = "".join([aa for aa in per_res_df["aa1"].astype(str).tolist() if aa in _CANONICAL_AA1])
    if not seq:
        return float("nan")
    try:
        return float(ProteinAnalysis(seq).isoelectric_point())
    except Exception:
        return float("nan")


def analyze_single_pdb(
    pdb_path: str,
    chain_id: Optional[str],
    hydrophobic_aa1: set,
) -> Tuple[MetricsResult, pd.DataFrame, str]:
    """Analyze one PDB (monomer chain). Returns (summary metrics, per-residue df, cleaned PDB text)."""
    structure = parse_pdb_to_structure(pdb_path)
    if chain_id is None or chain_id == "":
        chain_id = _first_protein_chain(structure)
        if chain_id is None:
            raise ValueError(f"No protein chain detected in: {os.path.basename(pdb_path)}")

    with tempfile.TemporaryDirectory() as td:
        clean_pdb = os.path.join(td, f"clean_{os.path.basename(pdb_path)}")
        write_clean_chain_pdb(structure, chain_id, clean_pdb)

        total_sasa, per_res = compute_sasa_with_freesasa(clean_pdb)
        per_res_aug = _augment_per_residue_df(clean_pdb, per_res, chain_id, hydrophobic_aa1)

        hydrophobic_sasa = float(per_res_aug.loc[per_res_aug["is_hydrophobic"], "sasa"].fillna(0.0).sum())
        hydrophobic_fraction = float(hydrophobic_sasa / total_sasa) if total_sasa > 0 else float("nan")
        predicted_pI = _predict_pi_from_df(per_res_aug)

        metrics = MetricsResult(
            filename=os.path.basename(pdb_path),
            chain_id=str(chain_id),
            total_sasa=float(total_sasa),
            hydrophobic_sasa=float(hydrophobic_sasa),
            hydrophobic_fraction=float(hydrophobic_fraction),
            predicted_pI=float(predicted_pI),
        )

        # Return cleaned PDB text (so viewer matches per_res_aug)
        with open(clean_pdb, "r", encoding="utf-8", errors="ignore") as f:
            clean_pdb_text = f.read()

        return metrics, per_res_aug, clean_pdb_text


def parse_hydrophobic_set(txt: str) -> set:
    s = set([c.strip().upper() for c in txt if c.strip()])
    valid = set(AA3_TO_AA1.values())
    return set([c for c in s if c in valid])

def generate_summary(all_results: Dict[str, dict], reference_filename: Optional[str] = None) -> None:
    if not all_results:
        print("No results to display.")
        return

    rows = []
    for fname, res in all_results.items():
        m = res["metrics"]
        per_res_df = res["per_res_df"]
        n = len(per_res_df)

        rows.append({
            "Filename": m.filename,
            "Chain": m.chain_id,
            "Num Residues": n,
            "Total SASA (Å²)": round(m.total_sasa, 2),
            "Hydrophobic SASA (Å²)": round(m.hydrophobic_sasa, 2),
            "Avg SASA/Residue (Å²)": round(m.total_sasa / n, 2) if n > 0 else 0.0,
            "Avg Hydrophobic SASA/Residue (Å²)": round(m.hydrophobic_sasa / n, 2) if n > 0 else 0.0,
            "Hydrophobic Fraction": round(m.hydrophobic_fraction, 2),
            "Predicted pI (ProtParam)": round(m.predicted_pI, 2) if not np.isnan(m.predicted_pI) else np.nan,
        })

    df = pd.DataFrame(rows)

    if reference_filename and reference_filename in all_results:
        ref_m = all_results[reference_filename]["metrics"]
        ref_df = all_results[reference_filename]["per_res_df"]
        ref_n = len(ref_df)

        ref_avg = ref_m.total_sasa / ref_n if ref_n > 0 else 0.0
        ref_avg_h = ref_m.hydrophobic_sasa / ref_n if ref_n > 0 else 0.0

        df["Δ Total SASA (Å²)"] = (df["Total SASA (Å²)"] - ref_m.total_sasa).round(2)
        df["Δ Hydrophobic SASA (Å²)"] = (df["Hydrophobic SASA (Å²)"] - ref_m.hydrophobic_sasa).round(2)
        df["Δ Avg SASA/Residue (Å²)"] = (df["Avg SASA/Residue (Å²)"] - ref_avg).round(2)
        df["Δ Avg Hydrophobic/Residue (Å²)"] = (df["Avg Hydrophobic SASA/Residue (Å²)"] - ref_avg_h).round(2)
        df["Δ Predicted pI"] = (df["Predicted pI (ProtParam)"] - round(ref_m.predicted_pI, 2)).round(2)

        ref_idx = df[df["Filename"] == reference_filename].index[0]
        ref_row = df.iloc[ref_idx:ref_idx+1]
        other = df.drop(ref_idx).sort_values("Hydrophobic SASA (Å²)", ascending=False)
        df = pd.concat([ref_row, other], ignore_index=True)
    else:
        df["Δ Total SASA (Å²)"] = 0.0
        df["Δ Hydrophobic SASA (Å²)"] = 0.0
        df["Δ Avg SASA/Residue (Å²)"] = 0.0
        df["Δ Avg Hydrophobic/Residue (Å²)"] = 0.0
        df["Δ Predicted pI"] = 0.0
        df = df.sort_values("Hydrophobic SASA (Å²)", ascending=False).reset_index(drop=True)

    display(df)


In [14]:
# @title Run Analysis
# @markdown Upload PDB file(s) and compute SASA metrics.

from google.colab import files  # <-- ensures files.upload() exists

hydrophobic_amino_acids = "ACFILMVWY"  # @param {type:"string"}
chain_id = ""  # @param {type:"string"}
enable_comparison = True  # @param {type:"boolean"}

ALL_RESULTS = {}
SINGLE_MODE_RESULTS = {}

print("Please upload your PDB file(s).")
uploaded = files.upload()

if not uploaded:
    print("No file uploaded.")
else:
    hydrophobic_set = parse_hydrophobic_set(hydrophobic_amino_acids)
    ALL_RESULTS.clear()

    for filename, content in uploaded.items():
        print(f"\nProcessing {filename}...")
        try:
            with tempfile.TemporaryDirectory() as td:
                pdb_path = os.path.join(td, filename)
                with open(pdb_path, "wb") as f:
                    f.write(content)

                target_chain = chain_id.strip() if chain_id.strip() else None

                metrics, per_res_df, pdb_text = analyze_single_pdb(
                    pdb_path=pdb_path,
                    chain_id=target_chain,
                    hydrophobic_aa1=hydrophobic_set,
                )

                entry = {
                    "metrics": metrics,
                    "per_res_df": per_res_df,
                    "pdb_text": pdb_text,  # <-- cleaned-chain PDB text
                    "chain_id": metrics.chain_id,
                    "filename": metrics.filename,
                }
                ALL_RESULTS[filename] = entry

                SINGLE_MODE_RESULTS.clear()
                SINGLE_MODE_RESULTS.update(entry)

                print(
                    f"Done. Chain: {metrics.chain_id}, "
                    f"Total SASA: {metrics.total_sasa:.1f}, "
                    f"Hydrophobic SASA: {metrics.hydrophobic_sasa:.1f}, "
                    f"Predicted pI: {metrics.predicted_pI:.2f}"
                )

        except Exception as e:
            print(f"Failed to analyze {filename}: {e}")
            traceback.print_exc()

    if enable_comparison and len(ALL_RESULTS) > 1:
        print("\nSelect a reference structure for delta calculations.")
        dropdown = widgets.Dropdown(
            options=list(ALL_RESULTS.keys()),
            description="Reference:",
            style={"description_width": "initial"},
            layout=widgets.Layout(width="500px"),
        )
        button = widgets.Button(
            description="Generate Summary with Deltas",
            button_style="success",
            layout=widgets.Layout(width="260px"),
        )
        output = widgets.Output()

        def on_button_click(_):
            output.clear_output(wait=True)
            with output:
                generate_summary(ALL_RESULTS, reference_filename=dropdown.value)
            button.disabled = True
            dropdown.disabled = True

        button.on_click(on_button_click)
        display(widgets.VBox([widgets.HBox([dropdown, button]), output]))
    else:
        generate_summary(ALL_RESULTS)


Please upload your PDB file(s).


Saving 1UBQ.pdb to 1UBQ.pdb

Processing 1UBQ.pdb...
Done. Chain: A, Total SASA: 4804.1, Hydrophobic SASA: 762.0, Predicted pI: 6.56


,Filename,Chain,Num Residues,Total SASA (Å²),Hydrophobic SASA (Å²),Avg SASA/Residue (Å²),Avg Hydrophobic SASA/Residue (Å²),Hydrophobic Fraction,Predicted pI (ProtParam),Δ Total SASA (Å²),Δ Hydrophobic SASA (Å²),Δ Avg SASA/Residue (Å²),Δ Avg Hydrophobic/Residue (Å²),Δ Predicted pI
0,1UBQ.pdb,A,76,4804.06,762.03,63.21,10.03,0.16,6.56,0.0,0.0,0.0,0.0,0.0


In [15]:
# @title Visualize Per-Residue SASA
# @markdown Select a structure to visualize its per-residue SASA profile

if len(ALL_RESULTS) == 0:
    print("No results available. Run the analysis cell first.")
else:
    # Create dropdown to select structure
    dropdown = widgets.Dropdown(
        options=list(ALL_RESULTS.keys()),
        description="Structure:",
        style={"description_width": "initial"},
        layout=widgets.Layout(width="500px"),
    )

    overlay_checkbox = widgets.Checkbox(
        value=False,
        description="Overlay all structures (max 5)",
        disabled=len(ALL_RESULTS) == 1,
    )

    button = widgets.Button(
        description="Plot",
        button_style="info",
        layout=widgets.Layout(width="100px"),
    )

    output = widgets.Output()

    def on_plot_click(_):
        output.clear_output(wait=True)
        with output:
            if overlay_checkbox.value and len(ALL_RESULTS) > 1:
                # Overlay mode
                plt.figure(figsize=(14, 5))
                for fname, res in list(ALL_RESULTS.items())[:5]:
                    per_res_df = res["per_res_df"].copy()
                    # Normalize residue numbering
                    per_res_df["normalized_resnum"] = range(1, len(per_res_df) + 1)
                    xs = per_res_df["normalized_resnum"].values
                    ys = per_res_df["sasa"].astype(float).values
                    plt.plot(xs, ys, label=fname, alpha=0.7, linewidth=1.5)

                plt.xlabel("Residue index (cleaned chain order)", fontsize=11)
                plt.ylabel("SASA (Å²)", fontsize=11)
                plt.title(
                    f"Per-residue SASA Comparison (n={min(5, len(ALL_RESULTS))})",
                    fontsize=12,
                    fontweight="bold",
                )
                plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left", fontsize=9)
                plt.grid(alpha=0.3)
                plt.tight_layout()
                plt.show()

            else:
                # Single structure mode
                selected = dropdown.value
                res = ALL_RESULTS[selected]
                per_res_df = res["per_res_df"].copy()

                # Normalize residue numbering
                per_res_df["normalized_resnum"] = range(1, len(per_res_df) + 1)
                xs = per_res_df["normalized_resnum"].values
                ys = per_res_df["sasa"].astype(float).values
                is_hydrophobic = per_res_df["is_hydrophobic"].astype(bool).values

                plt.figure(figsize=(14, 5))

                # Base line (light gray, continuous)
                plt.plot(xs, ys, color="lightgray", linewidth=1, alpha=0.6, zorder=1)

                # Colored scatter on top
                plt.scatter(
                    xs[is_hydrophobic],
                    ys[is_hydrophobic],
                    c="orange",
                    s=30,
                    alpha=0.85,
                    label="Hydrophobic",
                    zorder=2,
                )
                plt.scatter(
                    xs[~is_hydrophobic],
                    ys[~is_hydrophobic],
                    c="tab:blue",
                    s=30,
                    alpha=0.75,
                    label="Non-hydrophobic",
                    zorder=2,
                )

                plt.xlabel("Residue index (cleaned chain order)", fontsize=11)
                plt.ylabel("SASA (Å²)", fontsize=11)
                plt.title(f"Per-residue SASA: {selected}", fontsize=12, fontweight="bold")
                plt.legend(fontsize=9)
                plt.grid(alpha=0.3)
                plt.tight_layout()
                plt.show()

    button.on_click(on_plot_click)

    display(widgets.VBox([widgets.HBox([dropdown, overlay_checkbox, button]), output]))


In [16]:
# @title 3D Visualization


if not ALL_RESULTS:
    print("No results available. Run the analysis first.")
else:
    dropdown = widgets.Dropdown(
        options=list(ALL_RESULTS.keys()),
        description="Structure:",
        style={"description_width": "initial"},
        layout=widgets.Layout(width="520px")
    )

    show_hydrophobic_sticks = widgets.Checkbox(value=True, description="Show hydrophobics as sticks")
    color_hydrophobic_cartoon = widgets.Checkbox(value=True, description="Color hydrophobics in cartoon")
    show_hydrophobic_surface = widgets.Checkbox(value=False, description="Add hydrophobic surface (slower)")
    surface_opacity = widgets.FloatSlider(value=0.65, min=0.1, max=1.0, step=0.05, description="Surface opacity")

    out = widgets.Output()

    def render(_=None):
        out.clear_output(wait=True)
        with out:
            entry = ALL_RESULTS[dropdown.value]
            pdb_text = entry["pdb_text"]
            chain_id = entry["chain_id"]
            df = entry["per_res_df"]

            # Residue numbers for hydrophobic residues (unique, integer)
            hydro_resi = (
                df.loc[df["is_hydrophobic"], "resnum"]
                .dropna()
                .astype(int)
                .unique()
                .tolist()
            )

            view = py3Dmol.view(width=900, height=520)
            view.addModel(pdb_text, "pdb")

            # Base style for everything
            view.setStyle({"cartoon": {"color": "lightgray"}})

            # Highlight hydrophobics
            if hydro_resi:
                sel = {"chain": chain_id, "resi": hydro_resi}

                if color_hydrophobic_cartoon.value:
                    view.addStyle(sel, {"cartoon": {"color": "orange"}})

                if show_hydrophobic_sticks.value:
                    view.addStyle(sel, {"stick": {"color": "orange"}})

                if show_hydrophobic_surface.value:
                    # VDW surface for hydrophobic residues only (can be slow for large structures)
                    view.addSurface(
                        py3Dmol.VDW,
                        {"opacity": float(surface_opacity.value), "color": "orange"},
                        sel
                    )

            view.zoomTo()
            view.show()

    # Wire events
    dropdown.observe(render, names="value")
    show_hydrophobic_sticks.observe(render, names="value")
    color_hydrophobic_cartoon.observe(render, names="value")
    show_hydrophobic_surface.observe(render, names="value")
    surface_opacity.observe(render, names="value")

    display(widgets.VBox([
        dropdown,
        widgets.HBox([show_hydrophobic_sticks, color_hydrophobic_cartoon, show_hydrophobic_surface]),
        surface_opacity,
        out
    ]))

    render()


In [17]:
# @title Download Results

import zipfile

if not ALL_RESULTS:
    print("No results available. Run the analysis cell first.")
else:
    def add_single_to_zip(zf, entry, base_name=""):
        metrics = entry["metrics"]
        per_res_df = entry["per_res_df"]
        filename = entry["filename"]
        prefix = f"{base_name}/" if base_name else ""

        # 1) Per-residue SASA CSV
        buf = io.StringIO()
        per_res_df.to_csv(buf, index=False)
        zf.writestr(f"{prefix}{filename}_per_residue_sasa.csv", buf.getvalue())

        # 2) Summary CSV
        buf = io.StringIO()
        pd.DataFrame([metrics.__dict__]).to_csv(buf, index=False)
        zf.writestr(f"{prefix}{filename}_summary.csv", buf.getvalue())

    def download_all():
        zip_buffer = io.BytesIO()
        with zipfile.ZipFile(zip_buffer, "w", zipfile.ZIP_DEFLATED) as zf:
            for fname, entry in ALL_RESULTS.items():
                folder = fname.replace(".pdb","").replace(".ent","")
                add_single_to_zip(zf, entry, base_name=folder)

            # Comparative summary
            rows = []
            for fname, res in ALL_RESULTS.items():
                m = res["metrics"]
                n = len(res["per_res_df"])
                rows.append({
                    "Filename": m.filename,
                    "Chain": m.chain_id,
                    "Num Residues": n,
                    "Total SASA (Å²)": round(m.total_sasa, 2),
                    "Hydrophobic SASA (Å²)": round(m.hydrophobic_sasa, 2),
                    "Avg SASA/Residue (Å²)": round(m.total_sasa / n, 2) if n > 0 else 0.0,
                    "Avg Hydrophobic SASA/Residue (Å²)": round(m.hydrophobic_sasa / n, 2) if n > 0 else 0.0,
                    "Hydrophobic Fraction": round(m.hydrophobic_fraction, 4),
                    "Predicted pI (ProtParam)": round(m.predicted_pI, 2) if not np.isnan(m.predicted_pI) else np.nan,
                })
            summary_df = pd.DataFrame(rows).sort_values("Hydrophobic SASA (Å²)", ascending=False)
            buf = io.StringIO()
            summary_df.to_csv(buf, index=False)
            zf.writestr("comparative_summary.csv", buf.getvalue())

        zip_buffer.seek(0)
        out_name = "sasa_results.zip"
        with open(out_name, "wb") as f:
            f.write(zip_buffer.getvalue())
        files.download(out_name)
        print(f"Downloaded: {out_name}")

    download_all()


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Downloaded: sasa_results.zip
